# Average Articulation 
computed an average articulation of a disyllabic vowel utterances
- Created: 02 April 2020

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import librosa
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import librosa.display
from matplotlib import cm
from functools import partial

import sklearn
from sklearn.decomposition import PCA
from os import makedirs
import os
from os.path import join
import shutil
from IPython import display

%matplotlib inline

In [5]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
# np.random.seed(42)

In [8]:
EXP = '140_8'
# data_path = '../data/d_eval'
data_path = '../data/d_records/selected'

In [12]:
# path = '../experiment_2/result/eval_{}/testset.npz'.format(EXP)
path = '../experiment_2/result/predict_{}/testset.npz'.format(EXP)
data = np.load(path)['param_sets']
data = data.reshape((data.shape[0]*data.shape[1],24))
df = pd.DataFrame(data)

In [13]:
with open(join(data_path,'syllable_name.txt')) as f:
    syllable_name = np.array([word.strip() for line in f for word in line.split(',')])
    syllable_name = np.array([ '%s;%s'%(item,str(idx+1)) for pair in syllable_name for idx, item in enumerate(pair)])
    
df['phonetic'] = syllable_name
df['phonetic'], df['position'] = df['phonetic'].str.split(';', 1).str
df = df.groupby(['phonetic','position']).mean().reset_index()
df_1 = df[df['position']=='1'] 
df_2 = df[df['position']=='2'] 

In [14]:
df.head()

,phonetic,position,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,7,1,0.376734,-4.671421,0.0,-4.673518,0.316132,0.643250,0.815108,-0.047421,...,-0.478149,-6.696333,-2.371664,0.418178,0.323347,0.510548,0.053144,-0.05,-0.05,-0.05
1,7,2,0.318646,-4.534017,0.0,-4.465298,0.200810,0.794050,0.892573,-0.021735,...,-0.205849,-6.820206,-2.119418,0.507929,0.332670,0.701182,0.037012,-0.05,-0.05,-0.05
2,E,1,0.364068,-4.346879,0.0,-4.225955,-0.078352,0.942052,0.777071,0.020435,...,0.243491,-6.354609,-1.992054,1.102991,0.093639,0.305846,-0.374188,-0.05,-0.05,-0.05
3,E,2,0.352127,-4.152909,0.0,-4.323137,-0.133182,0.956194,0.796997,0.022142,...,0.268562,-6.351308,-1.954690,1.082736,0.117589,0.284179,-0.442249,-0.05,-0.05,-0.05
4,M,1,0.480736,-4.958945,0.0,-4.437208,0.621166,0.358467,0.401753,-0.044603,...,0.343493,-5.923912,-1.687408,1.077016,0.342870,0.295740,0.095920,-0.05,-0.05,-0.05


## Create Speaker File

In [15]:
def create_speaker_file(df, file_path):
                        
    speaker_header_file = '../experiment_2/lib/templates/speaker_template_head.txt'
    speaker_tail_file = '../experiment_2/lib/templates/speaker_template_tail.txt'

    # Read speaker template
    speaker_head = open(speaker_header_file,'r').read()
    speaker_tail = open(speaker_tail_file,'r').read()

    # load vocaltract param name
    param_name = np.load('../experiment_2/lib/templates/speaker_param.npz')['name']
    
    pho = set(df['phonetic'])
    
    with open(file_path, 'w') as f:
        f.write(speaker_head)

        for p in pho:
            df_temp = df[df['phonetic']==p].copy()
            f.write('<shape name="{}">\n'.format(p))
            for i in range(24):
                f.write('<param name="{}" value="{:.2f}"/>\n'.format(param_name[i],df_temp[i].values[0]))
            f.write('</shape>\n')

        f.write(speaker_tail)

In [17]:
create_speaker_file(df_1, 'speaker_140_pred_1.speaker')
create_speaker_file(df_2, 'speaker_140_pred_2.speaker')